# BMIN 5200 Foundations of Artificial Intelligence in Health

Assignment 3: Expert Systems in clipspy

In [3]:
import clips
import sys
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

In [2]:
# create the CLIPS environment
env = clips.Environment()

# Endometriosis Classification System

Endometriosis Classification Algorithm:
Using clipspy to implement an expert system that classifies a patient into one of the following categories:

Outputs:
1. Endometriosis
2. Not Endometriosis (some other disease causing symptoms)
3. Endometriosis and another concomitant disease

![resource](reference/clinical_diagnosis_algo.png)

# Templates

## Part 1: Defining Fact Templates (that are consistent with endometriosis)

This section defines and builds all of the templates that are needed to track and evalute facts associated with inclusion criteria.

In [10]:
# basic patient information
DEFTEMPLATE_PATIENT = """
(deftemplate patient 
    (slot name_is (type STRING))
    (slot age_is (type INTEGER))
    )
"""
env.build(DEFTEMPLATE_PATIENT)

# participant pelvic pain
DEFTEMPLATE_ACUTE_PELVIC_PAIN = """
(deftemplate pelvic_pain
    (slot has_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ACUTE_PELVIC_PAIN)

# participant Karnofsky quality of life score
DEFTEMPLATE_DYSMENORRHEA = """
(deftemplate dysmenorrhea
  (slot has_dysmenorrhea (type SYMBOL)
    (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_DYSMENORRHEA)

# participant HIV ELISA test result
DEFTEMPLATE_DYSPAREUNIA = """
(deftemplate dyspareunia
    (slot has_dyspareunia (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_DYSPAREUNIA)

# participant dyschezia status
DEFTEMPLATE_DYSCHEZIA = """
(deftemplate dyschezia
   (slot has_dyschezia (type SYMBOL)
        (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_DYSCHEZIA)

# participant dysuria status
DEFTEMPLATE_DYSURIA = """
(deftemplate dysuria
   (slot has_dysuria (type SYMBOL)
        (allowed-symbols yes no unknown)))
"""
env.build(DEFTEMPLATE_DYSURIA)

# participant pregnancy status
DEFTEMPLATE_INFERTILITY = """
(deftemplate infertility    
    (slot is_infertile (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_INFERTILITY)

# participant chronic pelvic pain
DEFTEMPLATE_CHRONIC_PELVIC_PAIN = """
(deftemplate chronic_pelvic_pain
    (slot has_chronic_pelvic_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_CHRONIC_PELVIC_PAIN)

## Part 2: Defining Fact Templates (factors indicating to consider another diagnossis in addition to OR instead of endometriosis)
Consistent with / not consistent wtih endo Fact Templates

In [ ]:
# participant has amenorrhea
DEFTEMPLATE_AMENORRHEA = """
(deftemplate amenorrhea
    (slot has_amenorrhea (type SYMBOL) 
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_AMENORRHEA)

# participant constipation status
DEFTEMPLATE_CONSTIPATION = """
(deftemplate constipation
    (slot has_constipation (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_CONSTIPATION)

# participant diarrhea status
DEFTEMPLATE_DIARRHEA = """
(deftemplate diarrhea
    (slot has_diarrhea (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_DIARRHEA)

# participant flank pain status
DEFTEMPLATE_FLANK_PAIN = """
(deftemplate flank_pain
    (slot has_flank_pain (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_FLANK_PAIN)

# participant hematuria status
DEFTEMPLATE_HEMATURIA = """
(deftemplate hematuria
    (slot has_hematuria (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_HEMATURIA)

# participant frequent urination status
DEFTEMPLATE_FREQUENT_URINATION = """
(deftemplate frequent_urination
    (slot has_frequent_urination (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_FREQUENT_URINATION)

# participant adenomyosis status
DEFTEMPLATE_ADENOMYOSIS = """
(deftemplate adenomyosis
    (slot has_adenomyosis (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ADENOMYOSIS)

## Consistent with / not consistent with endo Fact Templates
This section defines and builds all of the templates that are needed to track and evalute whether inclusion, exclusion, criteria for endometriosis and/or concomitant diseases.

In [ ]:
# participant inclusion criteria met status
DEFTEMPLATE_ENDOMETRIOSIS_INCLUSION = """
(deftemplate endometriosis_inclusion
    (slot meets_criteria (type SYMBOL)
        (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_ENDOMETRIOSIS_INCLUSION)

# participant trial eligibility status
DEFTEMPLATE_CONCOMITANT_DISEASE_INCLUSION = """
(deftemplate concomitant_disease_inclusion
    (slot meets_criteria (type SYMBOL)
    (allowed-symbols yes no unknown possible))
)
"""
env.build(DEFTEMPLATE_CONCOMITANT_DISEASE_INCLUSION)

# Initialize Knowledge Base

This section adds the necessary `deffacts`. These include four facts with slot values set to `unknown` to prevent some rules from firing at inappropriate times. As different rules are executed and facts asserted, these slot values will be updated so that, through chaining, other rules will then fire as needed. You should review the already implemented `defrules` carefully to understand how these work.

In [ ]:
# Add deffacts that the inclusion, exclusion, trial eligibility, and child bearing status are all unknown
DEFFCATS_INITIAL_STATUS = """
(deffacts starting_inclusion_exclusion_facts "Set the inclusion criteria met to unknown"
    (endometriosis_inclusion (meets_criteria unknown))
    (concomitant_disease_inclusion (meets_criteria unknown))
)
"""
env.build(DEFFCATS_INITIAL_STATUS)

# reset the environment to make sure the deffacts are added
env.reset()

# Inference Rules

## Inclusion Criteria Rules

The `defrules` in this section support gathering user input about the patient relative to the inclusion criteria and determining if the inclusion criteria are met. Notice that if the initial criteria (age, Karnofsky quality of life, life expectancy, and HIV status) are not met, no further information is collected as it is already known that trial eligibility will not be satisfied. If the initial inclusion criteria are met, then the value of the patient's sex is requested. If the patient is female, yet more information is requested.

In [ ]:
# These are the prompts that will be displayed to the user when requesting various template:slot inputs.
prompt_map = {
    "patient:name_is": "Enter patient name: ",
    "patient:age_is": "Enter patient age (in years): ",
    "pelvic_pain:has_pain": "Does the patient have acute pelvic pain (yes or no or unknown)?: ",
    "dysmenorrhea:has_dysmenorrhea": "Does the patient have dysmenorrhea (yes or no or unknown)?: ",
    "dyspareunia:has_dyspareunia": "Does the patient have dyspareunia (yes or no or unknown)?: ",
    "dyschezia:has_dyschezia" : "Does the patient have dyschezia (yes or no or unknown)?: ",
    "dysuria:has_dysuria" : "Does the patient have dysuria (yes or no or unknown)?: ",
    "infertility:is_infertile" : "Is the patient infertile (yes or no or unknown)?: ",
    "chronic_pelvic_pain:has_chronic_pelvic_pain" : "Does the patient have chronic pelvic pain (peritonitis) (yes or no or unknown)?: ",
    "amenorrhea:has_amenorrhea" : "Does the patient have amenorrhea (yes or no or unknown)?: ",
    "constipation:has_constipation" : "Does the patient have constipation related to IBS (yes or no or unknown)?: ",
    "diarrhea:has_diarrhea" : "Does the patient have diarrhea (yes or no or unknown)?: ",
    "flank_pain:has_flank_pain" : "Does the patient have flank pain (yes or no or unknown)?: ",
    "hematuria:has_hematuria" : "Does the patient have hematuria (yes or no or unknown)?: ",
    "frequent_urination:has_frequent_urination" : "Does the patient have frequent urination (yes or no or unknonw)?: ", 
    "adenomyosis:has_adenomyosis" : "Does the patient have adenomyosis (yes or no or unknown)?: "
}
build_read_assert(env, prompt_map) # see src/clips_util.py for implementation and demos/clips/clips_basics.ipynb for usage

DEFRULE_READ_INITIAL_INCLUSION = """
(defrule reading_input_initial
    =>
    (read_assert patient)
    (read_assert pelvic_pain)
    (read_assert dysmenorrhea)
    (read_assert dyspareunia)
    (read_assert dyschezia)
    (read_assert dysuria)
    (read_assert infertility)
    
)
"""
env.build(DEFRULE_READ_INITIAL_INCLUSION)

# # Determine child bearing age status using forward chaining based on female sex and age
# # Only want this to run if the female candidate is not already excluded based on other criteria.
# DEFRULE_CHILD_BEARING_AGE = """
# (defrule child_bearing_eval "Evaluate for child_bearing age or not for possibly eligible females"
#     (logical
#     	(patient (age_is ?age) (sex_is female))
#     	(test(>= ?age 18))
#     	(test(< ?age 45)) ;commonly accepted standard of upper limit of child bearing age of 45
    	
#     	(life_expect (years_expected_is ?life_expected))
#        (karnofsky (performance_status_is ?karnofsky_score))
#     	(test (>= ?life_expected 1))
#       	(test (>= ?karnofsky_score 60))
#  		(hiv_testing (hiv_elisa_is positive))
#  	)
    
#     =>
   
#     (assert (child_bearing(of_child_bearing_age yes)))
# )
# """
# env.build(DEFRULE_CHILD_BEARING_AGE)


# # Get additional information from user if it is determined that participant is a child bearing age female. 
# # Forward chaining to infer whether a female is still eligible by not being pregnant and having adequate pregnancy prevention
# DEFRULE_INPUT_PREG_EVAL = """
# (defrule reading_input_preg_eval "Ask for data on patient_pregnancy evaluation and birth control"
#         (child_bearing(of_child_bearing_age yes))

#         =>
        
#         (read_assert serum_preg)

# )
# """
# env.build(DEFRULE_INPUT_PREG_EVAL)


# # Get additional information from user if it is determined that participant is of child bearing age, female, and not already pregnant
# # Supports forward chaining to infer whether a female is still eligible by not being pregnant and having adequate pregnancy prevention
# DEFRULE_INPUT_BC = """
# (defrule reading_input_birthcontrol_eval "Ask for data on birth control if the patient is of child-bearing age and not pregnant"
#     (child_bearing(of_child_bearing_age yes))
#     (serum_preg (preg_test_is negative))
    
#     =>
    
#     ;(printout t "Is the patient on birth control (yes or no)?: ")
#     ;(assert (birth-control (on-birth-control (read))))

#     (read_assert birth_control)
# )
# """
# env.build(DEFRULE_INPUT_BC)

# ; RULE: Inclusion Criteria Are Not Met
# ; *Forward chaining to determine if participant is not eligible for study based on inclusion criteria facts
# ; INPUT: Criteria based on inclusion criteria defined in study. 
# ; OUTPUT: Trial eligibility No, Inclusion Criteria Met No
DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_NOT_MET = """
(defrule inclusion-criteria-notmet "Rule to define a person as not eligible for study based on inclusion criteria facts"
    (logical
        (and
            (pelvic_pain (has_pain ~yes))  
            (dysmenorrhea (has_dysmenorrhea ~yes))   
            (dyspareunia (has_dyspareunia ~yes)) 
            (dyschezia (has_dyschezia ~yes)) 
            (dysuria (has_dysuria ~yes)) 
            (infertility (is_infertile ~yes))   
            (chronic_pelvic_pain (has_chronic_pelvic_pain ~yes))
        )
    )

    ?f1 <-(endometriosis_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria no))
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_NOT_MET)

# indicates presence of symptom(s) that are consistent with endometriosis
DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_MET = """
(defrule inclusion-criteria-met "Rule to define a person as having symptom(s) consistent with endometriosis based on inclusion criteria facts"
    (logical
        (or
            (pelvic_pain (has_pain yes))  
            (dysmenorrhea (has_dysmenorrhea yes))   
            (dyspareunia (has_dyspareunia yes)) 
            (dyschezia (has_dyschezia yes)) 
            (dysuria (has_dysuria yes)) 
            (infertility (is_infertile yes))   
            (chronic_pelvic_pain (has_chronic_pelvic_pain yes))
        )
    )

    ?f1 <-(endometriosis_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria yes))
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_INCLUSION_CRITERIA_MET)

# indicates presence of symptoms that indicate no concomitant disease (along with or instead of) endometriosis
DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_NOT_MET = """
(defrule inclusion-criteria-not-met "Rule to define a person as having symptom(s) not consistent with other (non-endo) disease based on inclusion criteria facts"
    (logical
        (and
            (amenorrhea (has_amenorrhea ~yes))  
            (constipation (has_constipation ~yes))   
            (diarrhea (has_diarrhea ~yes)) 
            (flank_pain (has_flank_pain ~yes)) 
            (hematuria (has_hematuria ~yes)) 
            (frequent_urination (has_frequent_urination ~yes))   
            (adenomyosis (has_adenomyosis ~yes))
        )
    )

    ?f1 <-(concomitant_disease_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria no))
)
"""
env.build(DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_NOT_MET)

# indicates presence of symptoms that indicate concomitant disease (along with or instead of) endometriosis
DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_MET = """
(defrule inclusion-criteria-met "Rule to define a person as having symptom(s) consistent with other (non-endo) disease based on inclusion criteria facts"
    (logical
        (or
            (amenorrhea (has_amenorrhea yes))  
            (constipation (has_constipation yes))   
            (diarrhea (has_diarrhea yes)) 
            (flank_pain (has_flank_pain yes)) 
            (hematuria (has_hematuria yes)) 
            (frequent_urination (has_frequent_urination yes))   
            (adenomyosis (has_adenomyosis yes))
        )
    )

    ?f1 <-(concomitant_disease_inclusion (meets_criteria unknown))

    => 

    (modify ?f1 (meets_criteria yes))
)
"""
env.build(DEFRULE_CONCOMITANT_DISEASE_INCLUSION_CRITERIA_MET)
# not going to try and classify the concomitant disease - but will factor whether they do have a concomitant disease (IBS, or interstitial cystitis, or adenomyosis) in my evaluation
    # because I don't have detailed symptom data for all of the concomitant diseases, only some of them


## Inclusion Criteria Status Reporting Rules

The cell below defines `defrules` that fire once it is determine if the inclusion criteria are satisfied or not to report the status to the user.

In [14]:
DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_INCLUSION = """
(defrule endo-and-concoomitant-inclusion
    (endometriosis_inclusion (meets_criteria yes))
    (concomitant_disease_inclusion (meets_criteria yes))
    =>
    (println "___________")
    (println "Symptoms are consistent with both endometriosis and presence of concomitant disease. [recommendation]")
    (println "___________")
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_INCLUSION)

DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_EXCLUSION = """
(defrule endo-and-concoomitant-inclusion
    (endometriosis_inclusion (meets_criteria no))
    (concomitant_disease_inclusion (meets_criteria no))
    =>
    (println "___________")
    (println "Symptoms are consistent with neither endometriosis nor concomitant diseases that we screened for (IBS, adenomyosis, others). [recommendation]")
    (println "___________")
)
"""
env.build(DEFRULE_ENDOMETRIOSIS_AND_CONCOMITANT_EXCLUSION)

DEFRULE_ONLY_ENDOMETRIOSIS_INCLUSION = """
(defrule only-endo-inclusion
    (endometriosis_inclusion (meets_criteria yes))
    (concomitant_disease_inclusion (meets_criteria no))
    =>
    (println "___________")
    (println "Patient has symptom(s) consistent only with endometriosis and not additional diseases")
    (println "___________")
)
"""
env.build(DEFRULE_ONLY_ENDOMETRIOSIS_INCLUSION)

DEFRULE_ONLY_ENDOMETRIOSIS_EXCLUSION = """
(defrule only-endo-exclusion
    (endometriosis_inclusion (meets_criteria no))
    (concomitant_disease_inclusion (meets_criteria yes))
    =>
    (println "___________")
    (println "Patient has symptom(s) consistent only with non-endo diseases")
    (println "___________")
)
"""
env.build(DEFRULE_ONLY_ENDOMETRIOSIS_EXCLUSION)

# Execution
You should use the cells below to test the system behavior while implementing the `defrules` in this assignment. 

In [ ]:
env.reset();
env.run();

In [ ]:
print_facts(env)

## Evaluating Expert System Performance

In [ ]:
# three outputs
# calculate accuracy, PPV, NPV for each case

# and ablation analysis

# input data path: /project/ssverma_shared/projects/Endometriosis/Endo_RuleBased_Phenotyping/symptom_pulls/Pheno/PMBB_2.3_pheno_covars.csv